In [ ]:
import sys
import os
import cv2
import numpy as np
from PyQt5.QtWidgets import (QApplication, QMainWindow, QPushButton, QVBoxLayout,
                             QWidget, QLabel, QLineEdit, QMessageBox)
from PyQt5.QtCore import Qt, QTimer
from PyQt5.QtGui import QImage, QPixmap
from skimage import transform, color
from skimage.feature import hog
from skimage.color import rgb2gray
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC


class FacialRecognitionApp(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("Reconnaissance Faciale")
        self.setGeometry(100, 100, 400, 300)

        main_widget = QWidget()
        main_layout = QVBoxLayout()

        enregistrer_btn = QPushButton("Enregistrer un visage")
        enregistrer_btn.clicked.connect(self.open_enregistrement)
        analyser_btn = QPushButton("Analyser")
        analyser_btn.clicked.connect(self.open_analyse)

        main_layout.addWidget(enregistrer_btn)
        main_layout.addWidget(analyser_btn)
        main_widget.setLayout(main_layout)
        self.setCentralWidget(main_widget)

    def open_enregistrement(self):
        self.enregistrement_window = EnregistrementWindow()
        self.enregistrement_window.show()

    def open_analyse(self):
        self.analyse_window = AnalyseWindow()
        self.analyse_window.show()


class EnregistrementWindow(QWidget):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("Enregistrement de visage")
        self.setGeometry(200, 200, 500, 400)

        layout = QVBoxLayout()

        self.prenom_input = QLineEdit()
        self.prenom_input.setPlaceholderText("Prénom")
        self.nom_input = QLineEdit()
        self.nom_input.setPlaceholderText("Nom")

        valider_btn = QPushButton("Valider et Capturer")
        valider_btn.clicked.connect(self.validate_and_capture)

        layout.addWidget(QLabel("Prénom :"))
        layout.addWidget(self.prenom_input)
        layout.addWidget(QLabel("Nom :"))
        layout.addWidget(self.nom_input)
        layout.addWidget(valider_btn)

        self.setLayout(layout)

    def validate_and_capture(self):
        prenom = self.prenom_input.text()
        nom = self.nom_input.text()

        if not prenom or not nom:
            QMessageBox.warning(self, "Erreur", "Veuillez saisir un prénom et un nom")
            return

        self.capture_window = CaptureWindow(prenom, nom)
        self.capture_window.show()
        self.close()


class CaptureWindow(QWidget):
    def __init__(self, prenom, nom):
        super().__init__()
        self.prenom = prenom
        self.nom = nom
        self.setWindowTitle(f"Capture de {prenom} {nom}")
        self.setGeometry(200, 200, 640, 480)

        layout = QVBoxLayout()
        self.label = QLabel()
        layout.addWidget(self.label)

        self.enregistrer_btn = QPushButton("Terminer la capture")
        self.enregistrer_btn.setEnabled(False)  # Activé une fois le tour complet fait
        self.enregistrer_btn.clicked.connect(self.save_face)
        layout.addWidget(self.enregistrer_btn)

        self.setLayout(layout)

        self.capture = cv2.VideoCapture(0)
        self.timer = QTimer(self)
        self.timer.timeout.connect(self.update_frame)
        self.timer.start(30)

        self.face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
        self.captured_faces = []
        self.positions_captured = set()

    def update_frame(self):
        ret, frame = self.capture.read()
        if ret:
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            faces = self.face_cascade.detectMultiScale(gray, 1.1, 4)

            for (x, y, w, h) in faces:
                cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
                face_center = (x + w // 2, y + h // 2)
                self.positions_captured.add(face_center)
                self.captured_faces.append(frame[y:y+h, x:x+w])

            rgb_image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            h, w, ch = rgb_image.shape
            bytes_per_line = ch * w
            qt_image = QImage(rgb_image.data, w, h, bytes_per_line, QImage.Format_RGB888)
            pixmap = QPixmap.fromImage(qt_image)
            self.label.setPixmap(pixmap.scaled(640, 480, Qt.KeepAspectRatio))

            # Activer le bouton si suffisamment de visages/positions capturés
            if len(self.positions_captured) >= 10:  # Par exemple, 10 positions distinctes
                self.enregistrer_btn.setEnabled(True)

    def save_face(self):
        # Création du dossier spécifique à la personne
        person_dir = f'dataset/{self.prenom}_{self.nom}'
        os.makedirs(person_dir, exist_ok=True)

        # Sauvegarde des images capturées dans le sous-dossier
        for idx, face in enumerate(self.captured_faces):
            filename = f"{person_dir}/{self.prenom}_{self.nom}_{idx+1}.jpg"  # Index commence à 1
            cv2.imwrite(filename, face)

        QMessageBox.information(self, "Succès", f"Tour complet terminé. Visages enregistrés pour {self.prenom} {self.nom}")
        self.close()


class AnalyseWindow(QWidget):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("Analyse de visage")
        self.setGeometry(200, 200, 640, 480)

        layout = QVBoxLayout()
        self.label = QLabel()
        layout.addWidget(self.label)

        self.setLayout(layout)

        self.load_model()

        self.capture = cv2.VideoCapture(0)
        self.timer = QTimer(self)
        self.timer.timeout.connect(self.update_frame)
        self.timer.start(30)

        self.face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    def extract_features(self, image):
        image_gray = rgb2gray(image)
        image_resized = transform.resize(image_gray, (128, 64))
        features = hog(image_resized, orientations=9, pixels_per_cell=(16, 16), cells_per_block=(2, 2))
        return features

    def load_model(self):
        X = []
        y = []
        dataset_dir = '../dataset'

        # Vérifie si le dossier 'dataset' existe
        if not os.path.exists(dataset_dir):
            QMessageBox.warning(self, "Erreur", "Le dossier 'dataset' n'existe pas.")
            return

        # Parcours les sous-dossiers de 'dataset'
        for person_dir in os.listdir(dataset_dir):
            person_path = os.path.join(dataset_dir, person_dir)

            # Vérifie si c'est bien un dossier
            if os.path.isdir(person_path):
                # Parcours les fichiers dans chaque sous-dossier (nom de la personne)
                for filename in os.listdir(person_path):
                    if filename.endswith('.jpg'):
                        image_path = os.path.join(person_path, filename)
                        image = cv2.imread(image_path)

                        # Si l'image est lue correctement
                        if image is not None:
                            features = self.extract_features(image)
                            X.append(features)
                            y.append(person_dir)  # Utilise le nom du sous-dossier (personne)

        if len(X) == 0:
            QMessageBox.warning(self, "Erreur", "Aucun visage enregistré dans le dataset")
            return

        # Convertit les listes en tableaux numpy
        X = np.array(X)

        # Encode les labels (prénoms/noms) de manière numérique
        self.label_encoder = LabelEncoder()
        y_encoded = self.label_encoder.fit_transform(y)

        # Entraînement du classificateur SVM
        self.clf = SVC(kernel='linear', probability=True)
        self.clf.fit(X, y_encoded)


    def update_frame(self):
        ret, frame = self.capture.read()
        if ret:
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            faces = self.face_cascade.detectMultiScale(gray, 1.1, 4)

            for (x, y, w, h) in faces:
                face_roi = gray[y:y+h, x:x+w]
                face_resized = transform.resize(face_roi, (128, 64))

                face_features = hog(face_resized, orientations=9, pixels_per_cell=(16, 16), cells_per_block=(2, 2))

                try:
                    prediction = self.clf.predict([face_features])[0]
                    proba = self.clf.predict_proba([face_features])[0]
                    name = self.label_encoder.inverse_transform([prediction])[0]

                    if proba.max() > 0.7:
                        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
                        cv2.putText(frame, f"{name} ({proba.max():.2f})", (x, y-10),
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
                    else:
                        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 2)
                        cv2.putText(frame, "Inconnu", (x, y-10),
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)
                except Exception:
                    cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 2)
                    cv2.putText(frame, "Erreur modèle", (x, y-10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)

            rgb_image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            h, w, ch = rgb_image.shape
            bytes_per_line = ch * w
            qt_image = QImage(rgb_image.data, w, h, bytes_per_line, QImage.Format_RGB888)
            pixmap = QPixmap.fromImage(qt_image)
            self.label.setPixmap(pixmap.scaled(640, 480, Qt.KeepAspectRatio))


def main():
    app = QApplication(sys.argv)
    window = FacialRecognitionApp()
    window.show()
    sys.exit(app.exec_())


if __name__ == "__main__":
    main()
